# Question answering (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q kaggle

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"prakash56","key":"9ed53ea5e29b994ddcdff94a86fe2068"}'}

In [5]:
!mkdir ~/.kaggle

In [6]:
!cp kaggle.json ~/.kaggle/

In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d prokaggler22/oneapi-hackathon-the-llm-challenge


 81% 49.0M/60.5M [00:00<00:00, 89.2MB/s]

100% 60.5M/60.5M [00:00<00:00, 83.6MB/s]


In [9]:
!unzip oneapi-hackathon-the-llm-challenge.zip -d /content/oneapi-hackathon-the-llm-challenge

Archive:  oneapi-hackathon-the-llm-challenge.zip

  inflating: /content/oneapi-hackathon-the-llm-challenge/submission.csv  

  inflating: /content/oneapi-hackathon-the-llm-challenge/test.csv  

  inflating: /content/oneapi-hackathon-the-llm-challenge/train.csv  


In [2]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate

!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


You will need to setup git, adapt your email and name in the following cell.

In [17]:
!git config --global user.email "prakash.chem21@iitg.ac.in"
!git config --global user.name "Koo@12345"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [18]:
from huggingface_hub import notebook_login

notebook_login()

In [19]:
from datasets import load_dataset
import datasets

In [21]:
import pandas as pd
dfo = pd.read_csv("/kaggle/input/oneapi-hackathon-the-llm-challenge/train.csv")
dfo = dfo.dropna()
dfo = dfo.reset_index(drop=True)

In [22]:
from transformers import AutoTokenizer, AlbertForQuestionAnswering
import torch

model_checkpoint = "squirro/albert-base-v2-squad_v2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer.is_fast

True

In [ ]:
examples["question"]

NameError: name 'examples' is not defined

In [23]:
max_length = 512
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["Question"]]
    inputs = tokenizer(
        questions,
        examples["Story"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
# import pandas as pd
# df_train=pd.read_csv('/content/train.csv')
# df_test=pd.read_csv('/content/test.csv')

In [ ]:
dfo_train.columns

Index(['Story', 'Question', 'span_start', 'span_end', 'span_text', 'Answer'], dtype='object')

In [ ]:
df_train.iloc[0].question

'Who is the author of this series?'

In [ ]:
df_train.iloc[0].review

"Whether it be in her portrayal of a nerdy lesbian or a punk rock rebel, Maslany's plural personalities, (though very stereotypical), are entertaining eye-candy. Combined with a complex and unpredictable plot line, this show is surprisingly addictive. ANSWERNOTFOUND"

In [ ]:
df_train.iloc[0].human_ans_indices

'(251, 265)'

In [ ]:
df_train.iloc[0].review[251:265]

In [ ]:
df_train=df_train[['question','human_ans_indices','review','human_ans_spans']]
df_test=df_test[['question','human_ans_indices','review','human_ans_spans']]

In [24]:
dfo=dfo.drop(columns=['Answer'])


In [25]:
import numpy as np
dfo['id']=np.linspace(0,len(dfo)-1,len(dfo))
# df_test['id']=np.linspace(0,len(df_test)-1,len(df_test))

dfo['id']=dfo['id'].astype(str)
# df_test['id']=df_test['id'].astype(str)

In [ ]:
int(dfo_train.iloc[0].span_text.split('(')[1].split(',')[0])

IndexError: list index out of range

In [ ]:
float(df_train.iloc[0].human_ans_indices.split('(')[1].split(',')[1].split(' ')[1].split(')')[0])

265.0

In [ ]:
df_train['answers']=df_train['human_ans_spans']
df_test['answers']=df_test['human_ans_spans']

In [26]:
dfo['answers'] = dfo['span_text']

In [ ]:
dfo

,Story,Question,span_start,span_end,span_text,id,answers
0,CHAPTER IV. \n\nNotwithstanding the earnest in...,What emotions did she show?,199.0,288.0,The countenance of this young lady exhibited a...,0.0,{'text': ['The countenance of this young lady ...
1,"(CNN) -- Polk County, Florida, detectives arre...",What do police believe the motive to be?,428.0,465.0,We believe that his motive is robbery,1.0,{'text': ['We believe that his motive is robbe...
2,"Malawi (, or ; or [maláwi]), officially the Re...",Is it a large country?,653.0,702.0,Malawi is among the smallest countries in Africa.,2.0,{'text': ['Malawi is among the smallest countr...
3,CHAPTER XXXII. \n\nMR. GILMORE'S SUCCESS. \n\n...,What reminded him of Bullhampton?,1438.0,1448.0,Everything,3.0,"{'text': ['Everything'], 'answer_start': [1438]}"
4,CHAPTER XV--THE END OF THE MEETING \n\nStephen...,With what?,1465.0,1500.0,with feminine intuition she went on,4.0,{'text': ['with feminine intuition she went on...
...,...,...,...,...,...,...,...
44297,CHAPTER I--PROGRESS OF THE HOUSE \n\nThe winte...,Of what month?,709.0,713.0,Apri,44297.0,"{'text': ['Apri'], 'answer_start': [709]}"
44298,Hong Kong (CNN) -- A car theft in China that s...,What color?,1125.0,1145.0,the grey Toyota SUV,44298.0,"{'text': ['the grey Toyota SUV '], 'answer_sta..."
44299,CHAPTER III \n\nThe morning of departure arriv...,Who was crying?,505.0,581.0,"Eleanor, this should not be!"" said Reginald as...",44299.0,"{'text': ['Eleanor, this should not be!"" said ..."
44300,New York (CNN) -- Two New York men accused of ...,What did they do after beating him up?,661.0,728.0,stole Price's wallet and other personal proper...,44300.0,{'text': ['stole Price's wallet and other pers...


In [27]:
for i in range(0,len(dfo)):
  answer1={}
  si=int(dfo.iloc[i].span_start)
  ei=int(dfo.iloc[i].span_end)
  answer1['text']=[dfo.iloc[i].Story[si:ei]]
  answer1['answer_start']=[si]
  dfo.at[i,'answers']=answer1
  #print(df_train.iloc[i].answers,df_train.iloc[i].human_ans_spans)

In [ ]:
dfo['answers'].iloc[0]['text']

TypeError: string indices must be integers

In [ ]:
for i in range(0,len(df_test)):
  answer1={}
  si=int(df_test.iloc[i].human_ans_indices.split('(')[1].split(',')[0])
  ei=int(df_test.iloc[i].human_ans_indices.split('(')[1].split(',')[1].split(' ')[1].split(')')[0])
  answer1['text']=[df_test.iloc[i].review[si:ei]]
  answer1['answer_start']=[si]
  df_test.at[i, 'answers']=answer1
  #print(df_train.iloc[i].answers,df_train.iloc[i].human_ans_spans)

In [ ]:
dfo.columns

Index(['Story', 'Question', 'span_start', 'span_end', 'span_text', 'id',
       'answers'],
      dtype='object')

In [ ]:
df_train.columns=['question', 'human_ans_indices', 'context', 'human_ans_spans', 'id',
       'answers']

df_test.columns=['question', 'human_ans_indices', 'context', 'human_ans_spans','id',
       'answers']

In [ ]:
# import json

# # Convert the 'answers' column to JSON strings
# dfo['answers'] = dfo['answers'].apply(json.dumps)

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(dfo, test_size=0.2, random_state=42)

In [ ]:
df_train.iloc[0].answers

'"Why, Parson Harford have said over and over again, when he gave a shilling or so or a meat order, to help a poor lady that was ill, that \'twas by madam\'s wish"'

In [ ]:
train_dataset2

Dataset({
    features: ['Story', 'Question', 'span_start', 'span_end', 'span_text', 'id', 'answers', '__index_level_0__'],
    num_rows: 35442
})

In [29]:
val_dataset2 = datasets.Dataset.from_pandas(df_test)
train_dataset2 = datasets.Dataset.from_pandas(df_train)

In [30]:
train_dataset = train_dataset2.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=train_dataset2.column_names,
)
len(train_dataset2), len(train_dataset)

  0%|          | 0/36 [00:00<?, ?ba/s]

(35441, 35707)

In [23]:
train_dataset2.shape

(35441, 8)

In [31]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["Question"]]
    inputs = tokenizer(
        questions,
        examples["Story"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [32]:
validation_dataset = val_dataset2.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=val_dataset2.column_names,
)
len(validation_dataset)

  0%|          | 0/9 [00:00<?, ?ba/s]

8940

In [26]:
len(validation_dataset)

8940

In [27]:
validation_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 8940
})

In [28]:
len(val_dataset2)

8861

In [33]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [34]:
import torch
from transformers import AutoModelForQuestionAnswering

In [35]:
import collections

In [36]:
import evaluate

metric = evaluate.load("squad")

In [37]:
from tqdm.auto import tqdm


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["Story"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        start_index >= len(offsets) or start_index<0 or end_index<0
                        or end_index >= len(offsets)
                        or end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [38]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [37]:
# from huggingface_hub import notebook_login

# notebook_login()

In [43]:
from transformers import TrainingArguments

args = TrainingArguments(
    "/kaggle/working/oneapi-finetuned-run1",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=20,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=True,
)

In [44]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)


In [45]:
import numpy as np
n_best=20
max_answer_length = 30

In [38]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions

You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: ignored

In [ ]:
compute_metrics(start_logits, end_logits, validation_dataset, val_dataset2)

In [46]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.153200,No log
2,2.661200,No log
3,2.284600,No log
4,1.960400,No log
5,1.662600,No log
6,1.372000,No log
7,1.073500,No log
8,0.773400,No log
9,0.545200,No log
10,0.386900,No log


KeyboardInterrupt: 

In [48]:
import os
os.chdir(r'/kaggle/working')

!tar -czf oneapi-finetuned-run1.tar.gz oneapi-finetuned-run1

from IPython.display import FileLink

FileLink(r'oneapi-finetuned-run1.tar.gz')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/kaggle/working/oneapi-finetuned-run1.tar.gz

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

In [ ]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, val_dataset2)

In [ ]:
#Inference!

In [3]:
from transformers import pipeline

In [4]:
# Replace this with your own checkpoint
model_checkpoint2 = "/kaggle/input/oneapi-model/oneapi-finetuned-run1"
question_answerer = pipeline("question-answering", model=model_checkpoint2)

In [5]:
import pandas as pd
dfo = pd.read_csv('/kaggle/input/oneapi-hackathon-the-llm-challenge/test.csv')

In [6]:
dfo['span_text'] = dfo['Story']

In [ ]:
context = dfo.iloc[0].Story
question = dfo.iloc[0].Question
question_answerer(question=question, context=context)

In [18]:
from tqdm import tqdm
for i in tqdm(range(len(dfo))):
  context = dfo.iloc[i].Story
  question = dfo.iloc[i].Question
  dfo.at[i,'span_text'] = question_answerer(question=question, context=context)['answer']

100%|██████████| 28548/28548 [6:22:32<00:00,  1.24it/s]  


In [17]:
dfo

,Story,Question,span_text
0,"CHAPTER I \n\nON THE HOUSEBOAT \n\n""Say, Tom, ...",What was Sam's mood?,said Sam anxiously.
1,The Uralic languages (; sometimes called Urali...,By how many?,"approximately 25 million people,"
2,The University of Wisconsin–Madison (also know...,How many schools and colleges does it have?,UW–Madison is organized into 20 schools and co...
3,(CNN) -- The unexpected resignation of David P...,What happened?,unexpected resignation
4,"CHAPTER THIRTY-TWO \n\nTENDER TROUBLES \n\n""Jo...",Why does Mrs. March not force her children's c...,want
...,...,...,...
28543,"CHAPTER III. \n\n""Nice customs curt'sy to grea...",Who was the Archbishop of Toledo that was of t...,"CHAPTER III. \n\n""Nice customs curt'sy to grea..."
28544,CHAPTER XI. \n\nIn the little dining-room of t...,Who lived there?,CHAPTER XI. \n\nIn the little dining-room of t...
28545,"Chapter XXX. \n\n""I shall go on through all et...",is he fair?,"Chapter XXX. \n\n""I shall go on through all et..."
28546,Chapter XXXIV \n\nViolence \n\n\n\nIt had been...,Where was he to stay until this trip?,Chapter XXXIV \n\nViolence \n\n\n\nIt had been...


In [19]:
dfo.to_csv('new_test.csv')

In [5]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/bert-large-uncased-whole-word-masking-squad2"

In [ ]:
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
context = dfo['span_text'].iloc[0]
question = dfo['Question'].iloc[0]
QA_input = {
        'question': question,
        'context': context
}
print(nlp(QA_input)['answer'])

In [3]:
import pandas as pd
dfo = pd.read_csv('/kaggle/input/oneapi-temptest/new_test.csv')

In [4]:
dfout = []
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
for _, row in tqdm(dfo.iterrows(), total=len(dfo)):
    context = row['span_text']
    question = row['Question']
    QA_input = {
        'question': question,
        'context': context
    }
    res = nlp(QA_input)['answer']
    if res == '':
        QA_input = {
            'question': question,
            'context': row['Story']
        }
        res = nlp(QA_input)['answer']
    dfout.append(res)

NameError: name 'pipeline' is not defined

In [ ]:
dfou = pd.DataFrame(dfout, columns=['Answer'])

In [ ]:
dfou = dfou.replace('','hey')

In [ ]:
dfou.to_csv('submission.csv')

In [ ]:
import pandas as pd
df_train1=pd.read_csv('/content/train.csv')
df_test1=pd.read_csv('/content/test.csv')

In [ ]:
df_train1.iloc[13].question

'Why is the movie soo confusing?'

In [ ]:
context = df_train1.iloc[13].review
question = df_train1.iloc[13].question
question_answerer(question=question, context=context)

{'score': 0.16766905784606934,
 'start': 10,
 'end': 72,
 'answer': "is an interesting movie but might not be everyone's cup of tea"}

In [ ]:
# Replace this with your own checkpoint
model_checkpoint_o = "deepset/roberta-base-squad2"
question_answerer_old = pipeline("question-answering", model=model_checkpoint_o)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--deepset--roberta-base-squad2/snapshots/d39b8d4166b0683451bbce6f047de1a238c0b5bf/config.json

Model config RobertaConfig {

  "_name_or_path": "deepset/roberta-base-squad2",

  "architectures": [

    "RobertaForQuestionAnswering"

  ],

  "attention_probs_dropout_prob": 0.1,

  "bos_token_id": 0,

  "classifier_dropout": null,

  "eos_token_id": 2,

  "gradient_checkpointing": false,

  "hidden_act": "gelu",

  "hidden_dropout_prob": 0.1,

  "hidden_size": 768,

  "initializer_range": 0.02,

  "intermediate_size": 3072,

  "language": "english",

  "layer_norm_eps": 1e-05,

  "max_position_embeddings": 514,

  "model_type": "roberta",

  "name": "Roberta",

  "num_attention_heads": 12,

  "num_hidden_layers": 12,

  "pad_token_id": 1,

  "position_embedding_type": "absolute",

  "transformers_version": "4.25.1",

  "type_vocab_size": 1,

  "use_cache": true,

  "vocab_size": 50265

}



loading co

In [ ]:
context = df_train1.iloc[13].review
question = df_train1.iloc[13].question
question_answerer_old(question=question, context=context)

{'score': 0.4546004831790924,
 'start': 2649,
 'end': 2669,
 'answer': 'they were so orderly'}

In [ ]:
df_train.iloc[3].question

'Is this series good and excelent?'

In [ ]:
df_train.iloc[3].answers

{'text': ['this show is OUTSTANDING'], 'answer_start': [296]}

In [ ]:
df_train[['id','question','context','answers']].head()

,id,question,context,answers
0,0.0,Who is the author of this series?,Whether it be in her portrayal of a nerdy lesb...,"{'text': ['ANSWERNOTFOUND'], 'answer_start': [..."
1,1.0,Can we enjoy the movie along with our family ?,"An outstanding romantic comedy, 13 Going on 30...","{'text': ['ANSWERNOTFOUND'], 'answer_start': [..."
2,2.0,Does this one good?,"To let the truth be known, I watched this movi...","{'text': ['ANSWERNOTFOUND'], 'answer_start': [..."
3,3.0,Is this series good and excelent?,"At the time of my review, there had been 910 c...","{'text': ['this show is OUTSTANDING'], 'answer..."
4,4.0,How is the costume design?,"""Fright Night"" is great! This is how the story...",{'text': ['The costume design by Susan Matheso...


In [ ]:
len(df_train)

2501